In [2]:
import requests, time
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from collections import Counter
import functools
import operator
import string
from sklearn.model_selection import train_test_split

In [3]:
def get_categories():
    url = 'https://clientsfromhell.net/'
    html = requests.get(url)
    response = bs(html.content)
    get_items = [category for category in response.find_all('li', {'class':'flex items-center'})]
    categories = ['Dunces','Criminals','Deadbeats','Racists','Homophobes','Sexist','Frenemies','Cryptic','Ingrates','Chaotic Good']
    category_pair = []
    for item in get_items:
        href = item.find('a').get('href')
        item_name = re.sub('\\n','',item.text)
        pair = (item_name, href)
        if item_name in categories:
            category_pair.append(pair)
    return list(set(category_pair))

In [4]:
list_catetories = get_categories()
list_catetories

[('Dunces', '/tag/ignoramus/'),
 ('Ingrates', '/tag/Live-to-work/'),
 ('Racists', '/tag/Unwinnable-race/'),
 ('Homophobes', '/tag/Heterosapien/'),
 ('Cryptic', '/tag/Clear-as-coal/'),
 ('Sexist', '/tag/Breasts-and-Brains/'),
 ('Chaotic Good', '/tag/hell-michigan/'),
 ('Criminals', '/tag/freelance-felon/'),
 ('Frenemies', '/tag/Old-Friend/'),
 ('Deadbeats', '/tag/payday-mayday/')]

In [5]:
list_url_patters = []

def URL_CATEGORY_CREATOR(x):
    for cat in list_catetories:
        pattern = 'https://clientsfromhell.net'+cat[1]+'page/' # regex pattern for the urls to scrape
        list_url_patters.append((pattern,cat[0]))
    return list_url_patters
        
url_cat = URL_CATEGORY_CREATOR(list_catetories)
url_cat

[('https://clientsfromhell.net/tag/ignoramus/page/', 'Dunces'),
 ('https://clientsfromhell.net/tag/Live-to-work/page/', 'Ingrates'),
 ('https://clientsfromhell.net/tag/Unwinnable-race/page/', 'Racists'),
 ('https://clientsfromhell.net/tag/Heterosapien/page/', 'Homophobes'),
 ('https://clientsfromhell.net/tag/Clear-as-coal/page/', 'Cryptic'),
 ('https://clientsfromhell.net/tag/Breasts-and-Brains/page/', 'Sexist'),
 ('https://clientsfromhell.net/tag/hell-michigan/page/', 'Chaotic Good'),
 ('https://clientsfromhell.net/tag/freelance-felon/page/', 'Criminals'),
 ('https://clientsfromhell.net/tag/Old-Friend/page/', 'Frenemies'),
 ('https://clientsfromhell.net/tag/payday-mayday/page/', 'Deadbeats')]

In [6]:
for url in url_cat:
    print(url[0])

https://clientsfromhell.net/tag/ignoramus/page/
https://clientsfromhell.net/tag/Live-to-work/page/
https://clientsfromhell.net/tag/Unwinnable-race/page/
https://clientsfromhell.net/tag/Heterosapien/page/
https://clientsfromhell.net/tag/Clear-as-coal/page/
https://clientsfromhell.net/tag/Breasts-and-Brains/page/
https://clientsfromhell.net/tag/hell-michigan/page/
https://clientsfromhell.net/tag/freelance-felon/page/
https://clientsfromhell.net/tag/Old-Friend/page/
https://clientsfromhell.net/tag/payday-mayday/page/


In [7]:
def PAGE_NUM_CREATOR(x):
    list_url_num =[]
    for url in url_cat:
        html = requests.get(url[0]+'1')
        response = bs(html.content)
        list_items = response.find_all('a',{'class':'page-numbers'})

        len_=len(list_items)-2
        max_pag=list_items[len_].text
        list_url_num.append((url[0],max_pag,url[1]))
    return list_url_num

list_all = PAGE_NUM_CREATOR(url_cat)
list_all

[('https://clientsfromhell.net/tag/ignoramus/page/', '90', 'Dunces'),
 ('https://clientsfromhell.net/tag/Live-to-work/page/', '27', 'Ingrates'),
 ('https://clientsfromhell.net/tag/Unwinnable-race/page/', '7', 'Racists'),
 ('https://clientsfromhell.net/tag/Heterosapien/page/', '3', 'Homophobes'),
 ('https://clientsfromhell.net/tag/Clear-as-coal/page/', '55', 'Cryptic'),
 ('https://clientsfromhell.net/tag/Breasts-and-Brains/page/', '6', 'Sexist'),
 ('https://clientsfromhell.net/tag/hell-michigan/page/', '44', 'Chaotic Good'),
 ('https://clientsfromhell.net/tag/freelance-felon/page/', '15', 'Criminals'),
 ('https://clientsfromhell.net/tag/Old-Friend/page/', '4', 'Frenemies'),
 ('https://clientsfromhell.net/tag/payday-mayday/page/', '64', 'Deadbeats')]

In [8]:
class IronhackSpider:
    
    def __init__(self, url_pattern, pages_to_scrape=1, sleep_interval=-1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
  
    def scrape_url(self, url):
        response = requests.get(url)
        result = self.content_parser(response.content)
        return result
#         self.list_results(result)
        
#     def list_results(self, r):
#         list_pages = []
#         list_pages.append(r)
#         return list_pages

#     def output_results(self, r):
#         print(r)

#     def kickstart(self):
#         for i in range(1, self.pages_to_scrape+1):
#             self.scrape_url(self.url_pattern % i)
            
    def kickstart(self):
        list_pages = []
        for i in range(1, self.pages_to_scrape+1):
            list_pages.append(self.scrape_url(self.url_pattern % i))            
        return list_pages


def content_parser(content):
    return content

def case_parser(content):
    all_content = bs(content, "html.parser")
    pre_content = all_content.select('div [class="w-blog-post-content"] > p')
    
    case=[]
    
    for i, el in enumerate(pre_content):
        text = el.text
        case.append(text)

    return case

# Instantiate the IronhackSpider class
html_cont_dict = {}

for URL_PATTERN, PAGES_TO_SCRAPE, CAT in list_all:
#     print(URL_PATTERN, PAGES_TO_SCRAPE, CAT)
    my_spider = IronhackSpider(URL_PATTERN+'%s/', int(PAGES_TO_SCRAPE), content_parser=case_parser)
    # Start scraping jobs
    content = my_spider.kickstart()
    html_cont_dict.update({CAT: content})
    
#     html_cont_dict.update(CAT, my_spider.kickstart())
    
html_cont_dict['Frenemies']

o walk around and experiment how my\nlocations will look. I also need to practice filming.Me:\xa0No, I have to take it with me.Client:\xa0Okay, fine. Wow, I guess you’re really attached to your camera!\xa0',
  'He had a way of suggesting my needs were personal faults. I wouldn’t have been surprised if I asked to be paid, and he responded\xa0“I guess some people are just caught in the rat race, man!”',
  'I accepted a small job of making a flyer for my\naunt’s boyfriend’s amateur jazz band. After three weeks of going back and\nforth, unnecessary changes that could have been avoided easily had his first\nbrief been complete, I send him the invoice.',
  'I get a call from him the next day.',
  'Client: Well I didn’t know it would cost this much\nmoney!',
  'Me: I’m doing this work at a pretty big discount\nactually.',
  'Client: Look, I asked you because we’re\nbasically family! I wanted to give you some work! I mean I could’ve\ngone to anyone else!',
  'Never again am I working for “fami

In [11]:
import csv
with open('client_from_hell_dic.csv', 'w') as f:
    for key in html_cont_dict.keys():
        f.write("%s,%s\n"%(key,html_cont_dict[key]))

In [12]:
stopwords = ["mustn't", 'have', 'now', "shouldn't", 'down', 'theirs', 
             'where', 'doing', 'who', 'is', 'how', 'which', 'up', 'if', 
             'nor', 'y', 'a', 'couldn', "weren't", 'your', 'in', 'themselves', 
             'only', 'both', 'will', 'own', 'we', 'by', 'these', 'into', 'once', 
             'her', 'when', 'o', 's', 've', 'again', 'mustn', 'they', 't', 'aren', 
             'his', "haven't", "won't", 'same', 'not', 'there', 'hadn', 'has', 
             'can', "didn't", 'those', 'then', 'this', 'shan', 'such', 'wasn', 
             'of', 'after', 'through', 'from', "you'd", 'shouldn', "isn't", 'being', 
             'too', "that'll", 'between', 'had', 'yours', 'very', 'i', 'because', 
             'him', 'are', "needn't", 'before', 'wouldn', 'my', 'what', 'the', 'during', 
             'isn', 'weren', 'against', 'don', 'off', 'll', 'am', 'our', 'should', 
             "should've", "you've", 'hers', 'their', 'more', 'than', 'himself', 'itself', 
             'with', 'just', "hasn't", 'he', 'any', 'to', "couldn't", "you'll", 'on', 'she', 
             "she's", 'ma', 'an', 'haven', 'was', 'few', 'each', 'that', 'further', 'didn', 
             "hadn't", "it's", 'over', 'you', 'me', 'about', 'mightn', 'most', 'be', 'so', 
             'won', 'them', 'why', 'needn', 'or', 'above', "doesn't", 'd', 'yourselves', 
             'ours', 'herself', 'as', 'some', 'under', "don't", 'at', 'all', "wouldn't", 'it', 
             'does', 'but', 'below', 'here', 'other', 'whom', 'ourselves', "you're", 'did', 'do', 
             'while', "aren't", 'having', 're', 'ain', 'were', 'hasn', 'no', 'out', 'and', 'for', 
             'yourself', 'been', 'm', 'myself', 'its', 'until', "shan't", "wasn't", "mightn't", 'doesn', 
             "you’re", "aren’t"]

test = "you're"

test not in stopwords

False

In [13]:
df_clients_og = pd.DataFrame.from_dict(html_cont_dict, orient = 'index').fillna('')
df_clients_og = df_clients_og.T

def stem(a):
    p = PorterStemmer()
    a = [p.stem(word) for word in a]
    return a

# df_clients_og = pd.DataFrame.from_dict(html_cont_dict, orient = 'index').fillna('')
# df_clients_og = df_clients_og.T

for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list=[]

        for item in list_:
            if item.startswith('Client:'):
                sub_list.append(item)

        df_clients_og[col][i] = sub_list
#         print(sub_list)
        
punc_list = [x for x in string.punctuation]

for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list = [x.replace('\xa0', ' ') for x in df_clients_og[col][i]]
        sub_list = [x.replace('\n', ' ') for x in sub_list]
        sub_list = [x.replace('Client: ', '') for x in sub_list]
        for punc in punc_list:
            sub_list = [x.replace(punc, '') for x in sub_list]
        sub_list = [x.replace('—', '') for x in sub_list]
        sub_list = [x.replace('   ', ' ') for x in sub_list]
        sub_list = [x.replace('  ', ' ') for x in sub_list]
        sub_list = [x.rstrip() for x in sub_list]
        
        df_clients_og[col][i] = sub_list
        
for col in df_clients_og:
#     print(col)
        
    for i,list_ in enumerate(df_clients_og[col]):
        sub_list = [x.split(' ') for x in list_]
        
        df_clients_og[col][i] = sub_list
        df_clients_og[col][i] = [word.lower() for words in df_clients_og[col][i] for word in words if len(word) != 1]
        df_clients_og[col][i] = [re.sub(r'^(.)\1+', r'\1', word)  for word in df_clients_og[col][i]]

        df_clients_og[col][i] = [word.replace("’", "'") for word in df_clients_og[col][i]]
#         df_clients_og[col][i] = [word.replace("ing", "") for word in df_clients_og[col][i]]
#         df_clients_og[col][i] = [word.replace("ed", "") for word in df_clients_og[col][i]]
        df_clients_og[col][i] = [word.rstrip("'") for word in df_clients_og[col][i]]
        
        
        
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if word not in stopwords]
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if word.isalpha() == True]
        df_clients_og[col][i] = [word for word in df_clients_og[col][i] if len(word) != 1]
        df_clients_og[col][i] = stem(df_clients_og[col][i])
        
        print(df_clients_og[col][i])
        
        df_clients_og[col][i] = dict(Counter(df_clients_og[col][i]))

ed', 'sort', 'want', 'fix', 'great', 'thank', 'good', 'job', 'brilliant', 'send', 'invoic', 'show', 'client', 'like', 'realli', 'chang', 'agre', 'bit', 'naff', 'realli', 'pay', 'call', 'account', 'depart', 'great', 'want', 'someth', 'look', 'like', 'everyth', 'els', 'categori', 'make', 'pay', 'ruin', 'relationship', 'oh', 'sure', 'almost', 'forgot', 'much', 'owe', 'wait', 'thought', 'one', 'receiv', 'money', 'come', 'tell', 'actual', 'pay', 'cours', 'think', 'kind', 'short', 'us', 'take', 'away', 'first', 'last', 'paragraph', 'word', 'togeth', 'word', 'like', 'build', 'websit', 'compani', 'abl', 'pay', 'month', 'start', 'make', 'money', 'shame', 'year', 'thought', 'friend', 'probabl', 'abl', 'afford', 'type', 'invoic', 'next', 'month', 'thing', 'pretti', 'tight', 'cash', 'flow', 'right', 'still', 'need', 'amount', 'work', 'complet', 'report', 'thing', 'go', 'pay', 'half', 'much', 'sorri', 'money', 'money']
['client', 'unhappi', 'peopl', 'take', 'part', 'campaign', 'sure', 'pay', 'sinc'

In [18]:
all_keys = []

for col in df_clients_og:
#     print(col)
        
    for i,dic_ in enumerate(df_clients_og[col]):
        for word in list(dic_.keys()):
            all_keys.append(word)

all_keys = list(set(all_keys))

# print(len(all_keys))
    
all_keys = sorted(all_keys)
# print(all_keys)
        

In [19]:
df_final = df_clients_og.T

df_final.columns = [str(col) for col in df_final.columns]

df_final.reset_index(inplace = True)
df_final.rename(columns = {'index':'category'}, inplace = True)
# df_final.head()

df_cases = pd.DataFrame(columns = ['category', 'case'])

# df_final[['category', '0']].rename(columns = {'0':'case'})

for col in df_final:
    if col != 'category':
        df_cases = df_cases.append(df_final[['category', col]].rename(columns = {col:'case'}))

df_cases.reset_index(drop = True, inplace = True)

for i, row in enumerate(df_cases['case']):
    if row == {}:
        df_cases.drop(index = i, inplace = True)
        
df_cases.reset_index(drop = True, inplace = True)
df_cases


,category,case
0,Dunces,"{'start': 1, 'new': 1, 'project': 1, 'someth':..."
1,Ingrates,"{'pixel': 1, 'perfect': 1, 'absolut': 2, 'garb..."
2,Racists,"{'poster': 1, 'beauti': 1, 'im': 1, 'afraid': ..."
3,Homophobes,"{'loud': 1, 'proud': 1, 'gay': 1, 'understand'..."
4,Cryptic,"{'like': 10, 'want': 8, 'white': 2, 'backgroun..."
...,...,...
309,Dunces,"{'name': 1, 'sort': 1, 'stupid': 1, 'introduc'..."
310,Dunces,"{'finish': 3, 'product': 2, 'seem': 1, 'exactl..."
311,Dunces,"{'decid': 2, 'want': 1, 'call': 1, 'new': 1, '..."
312,Dunces,"{'outrag': 1, 'last': 1, 'crew': 1, 'could': 1..."


In [21]:
df_count = pd.DataFrame()

for i in range(df_cases['case'].shape[0]):
#     print(i)
    df_count = df_count.append(pd.DataFrame(df_cases['case'][i], index=[0]))
    
df_count.reset_index(drop = True, inplace = True)
df_count.head()

# df_count.fillna('0', inplace = True)


# df_count = pd.DataFrame(df_cases['case'][0], index=[0])

# df_count.append(pd.DataFrame(df_cases['case'][1], index=[0]))

# df_count

,start,new,project,someth,big,talk,youtub,specif,market,money,...,centr,getgo,georg,webinar,foot,advic,arrow,huff,woah,phrase
0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_count.shape[0]

314

In [24]:
df_count['category'] = df_cases['category']
df_count.to_csv('final_dataframe.csv')
df_count.head()

,start,new,project,someth,big,talk,youtub,specif,market,money,...,getgo,georg,webinar,foot,advic,arrow,huff,woah,phrase,category
0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dunces
1,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ingrates
2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Racists
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Homophobes
4,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cryptic


In [28]:
df_up = pd.read_csv('final_dataframe.csv')
df_up.drop(columns = 'Unnamed: 0', inplace = True)
df_final = df_final.fillna('0')

In [29]:
features = df_final[[col for col in df_final.columns if col != 'category']]
features = features.astype(int)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314 entries, 0 to 313
Columns: 4061 entries, start to phrase
dtypes: int64(4061)
memory usage: 9.7 MB


In [31]:
target = df_final[['category']]
#target

for i in range(target.shape[0]):
    target['category'] = [x.replace(x, '1') if x == 'Deadbeats' else x.replace(x,'0') for x in target['category']]
    
target = target.astype(int)
# type(target['category'][100])

target['category'][0]

0

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    features, # Features (X)
    target, # Target (y)
    test_size = .2,
    random_state = 42
)

X_train.head()

,start,new,project,someth,big,talk,youtub,specif,market,money,...,centr,getgo,georg,webinar,foot,advic,arrow,huff,woah,phrase
113,2,1,1,1,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
101,0,0,0,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
137,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,0,3,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,0,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
y_train.head()

,category
113,0
101,0
137,0
73,0
93,0


In [34]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3).fit(X_train, y_train.values.ravel())

knn.score(X_test, y_test)

1.0

In [35]:
from sklearn.naive_bayes import BernoulliNB

bernoulli = BernoulliNB().fit(X_train, y_train.values.ravel())
bernoulli.score(X_test, y_test)

1.0

In [36]:
from sklearn.naive_bayes import GaussianNB

guassian = GaussianNB().fit(X_train, y_train.values.ravel())
guassian.score(X_test, y_test)

1.0

In [37]:
def model_score_mul(x):
    import time
    
    score = x.score(X_test, y_test_mul)*100
    if score >= 50.0:
        print('The score for this algorithm is (drum roll please!!!): ')
        time.sleep(3)
        print('.... processing')
        time.sleep(3)
        print('....')
        time.sleep(3)
        print('*looks at you confused*')
        time.sleep(3)
        print('oh yeah, you are waiting for the score... LOL')
        print('score: ',score,'%')
        print("DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!")
    elif score < 50.0:
        print('I got good news, and bad news...')
        time.sleep(2)
        print('the bad news is that you will not like the score...')
        time.sleep(2)
        print("Now, the good news is that I'm right here to watch your face!!! LOL")
        print('The score for this algorithm is: ',score,'%')
        time.sleep(2)
        print('')
        print("Your algorithm stinks so much, I could toss a coin and make better predictions =P...")

In [38]:
cat_dic = {'Dunces': 0,'Criminals': 1,'Deadbeats': 2,'Racists': 3,'Homophobes': 4,
        'Sexist': 5,'Frenemies': 6,'Cryptic': 7,'Ingrates': 8,'Chaotic Good': 9}

In [40]:
target_mul = df_final[['category']].replace(cat_dic, regex=True)
# target_mul.head()

# df['commentTest'] = df['Comment'].replace(ids, regex=True)

X_train, X_test, y_train_mul, y_test_mul = train_test_split(
    features, # Features (X)
    target_mul, # Target (y)
    test_size = .2,
    random_state = 42
)

from sklearn.linear_model import LogisticRegression

log_regr = LogisticRegression(solver = 'lbfgs')
log_regr.fit(X_train, y_train_mul.values.ravel())
model_score_mul(log_regr)

The score for this algorithm is (drum roll please!!!): 
.... processing
....
*looks at you confused*
oh yeah, you are waiting for the score... LOL
score:  57.14285714285714 %
DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!


In [41]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 3) # k = 5 by default
knn.fit(X_train, y_train_mul.values.ravel())
model_score_mul(knn)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  44.44444444444444 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [42]:
from sklearn.preprocessing import RobustScaler

scalar = RobustScaler().fit(X_train)
scalar.transform(X_train)[:5]

array([[2., 1., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [43]:
knn_robust = KNeighborsClassifier()
knn_robust.fit(scalar.transform(X_train), y_train_mul.values.ravel())
model_score_mul(knn_robust)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  47.61904761904761 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [44]:
from sklearn.naive_bayes import MultinomialNB

multi = MultinomialNB()
multi.fit(X_train, y_train_mul.values.ravel())
model_score_mul(multi)

The score for this algorithm is (drum roll please!!!): 
.... processing
....
*looks at you confused*
oh yeah, you are waiting for the score... LOL
score:  53.96825396825397 %
DON'T GET COCKY NOW!!! KEEP MAKING IT BETTER!


In [45]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=10, random_state=42)
rfc.fit(X_train, y_train_mul.values.ravel())
model_score_mul(rfc)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  44.44444444444444 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [48]:
features_bool = df_final[[col for col in df_final.columns if col != 'category']]
features_bool = features.astype(bool)

In [49]:
features_bool.head()

,start,new,project,someth,big,talk,youtub,specif,market,money,...,centr,getgo,georg,webinar,foot,advic,arrow,huff,woah,phrase
0,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [50]:
X_train_bool, X_test_bool, y_train_mul, y_test_mul = train_test_split(
    features_bool, # Features (X)
    target_mul, # Target (y)
    test_size = .2,
    random_state = 42
)

X_train_bool.head()

,start,new,project,someth,big,talk,youtub,specif,market,money,...,centr,getgo,georg,webinar,foot,advic,arrow,huff,woah,phrase
113,True,True,True,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
101,False,False,False,True,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
137,True,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
73,False,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
y_train_mul

,category
113,2
101,1
137,9
73,0
93,7
...,...
188,2
71,1
106,9
270,0


In [52]:
bernoulli = BernoulliNB().fit(X_train_bool, y_train_mul.values.ravel())
model_score_mul(bernoulli)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  33.33333333333333 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...


In [53]:
pred_log_regr = log_regr.predict(X_test)
pred_multi = multi.predict(X_test)
pred_bernoulli = bernoulli.predict(X_test_bool)


In [54]:
from sklearn.metrics import r2_score

print('R^2 for Log Regression: ', r2_score(y_test_mul, pred_log_regr))
print('R^2 for Multinomial NB: ', r2_score(y_test_mul, pred_multi))
print('R^2 for Bernoulli NB: ', r2_score(y_test_mul, pred_bernoulli))


R^2 for Log Regression:  -0.3856222311719695
R^2 for Multinomial NB:  -0.852194925493355
R^2 for Bernoulli NB:  -1.5443012484897305


In [55]:
from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(linear['actual'], linear['predicted'])

print('MSE for Log Regression: ', mean_squared_error(y_test_mul, pred_log_regr))
print('MSE for Multinomial NB: ', mean_squared_error(y_test_mul, pred_multi))
print('MSE for Bernoulli NB: ', mean_squared_error(y_test_mul, pred_bernoulli))


MSE for Log Regression:  15.603174603174603
MSE for Multinomial NB:  20.857142857142858
MSE for Bernoulli NB:  28.650793650793652


In [56]:
from math import sqrt

print('RMSE for Log Regression: ', sqrt(mean_squared_error(y_test_mul, pred_log_regr)))
print('RMSE for Multinomial NB: ', sqrt(mean_squared_error(y_test_mul, pred_multi)))
print('RMSE for Bernoulli NB: ', sqrt(mean_squared_error(y_test_mul, pred_bernoulli)))

RMSE for Log Regression:  3.9500853918839023
RMSE for Multinomial NB:  4.566962103755937
RMSE for Bernoulli NB:  5.3526436132806055


In [57]:
from sklearn.metrics import mean_absolute_error

# mean_absolute_error(linear['actual'], linear['predicted'])

print('MAE for Log Regression: ', mean_absolute_error(y_test_mul, pred_log_regr))
print('MAE for Multinomial NB: ', mean_absolute_error(y_test_mul, pred_multi))
print('MAE for Bernoulli NB: ', mean_absolute_error(y_test_mul, pred_bernoulli))

MAE for Log Regression:  2.2063492063492065
MAE for Multinomial NB:  2.8253968253968256
MAE for Bernoulli NB:  3.9523809523809526


In [58]:
from sklearn.metrics import accuracy_score

# accuracy_score(classifier['actual'], classifier['predicted'])

print('Accuracy score for Log Regression: ', accuracy_score(y_test_mul, pred_log_regr))
print('Accuracy score for Multinomial NB: ', accuracy_score(y_test_mul, pred_multi))
print('Accuracy score for Bernoulli NB: ', accuracy_score(y_test_mul, pred_bernoulli))

Accuracy score for Log Regression:  0.5714285714285714
Accuracy score for Multinomial NB:  0.5396825396825397
Accuracy score for Bernoulli NB:  0.3333333333333333


In [59]:
from sklearn.metrics import balanced_accuracy_score

# balanced_accuracy_score(classifier['actual'], classifier['predicted'])

print('Balanced accuracy score for Log Regression: ', balanced_accuracy_score(y_test_mul, pred_log_regr))
print('Balanced accuracy score for Multinomial NB: ', balanced_accuracy_score(y_test_mul, pred_multi))
print('Balanced accuracy score for Bernoulli NB: ', balanced_accuracy_score(y_test_mul, pred_bernoulli))

Balanced accuracy score for Log Regression:  0.31063112745098037
Balanced accuracy score for Multinomial NB:  0.30603553921568627
Balanced accuracy score for Bernoulli NB:  0.1953125


In [60]:
inv_cat_dic = dict(map(reversed, cat_dic.items()))
inv_cat_dic

{0: 'Dunces',
 1: 'Criminals',
 2: 'Deadbeats',
 3: 'Racists',
 4: 'Homophobes',
 5: 'Sexist',
 6: 'Frenemies',
 7: 'Cryptic',
 8: 'Ingrates',
 9: 'Chaotic Good'}

In [61]:
from sklearn.metrics import confusion_matrix

confusion = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_log_regr),
)

confusion.rename(columns = inv_cat_dic, inplace = True)
confusion.rename(index = inv_cat_dic, inplace = True)

print('Log Regression')
confusion

Log Regression


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic
Dunces,8,0,1,0,0,0,2,1
Criminals,1,0,2,0,0,0,0,1
Deadbeats,3,0,13,0,0,0,0,0
Racists,0,0,0,0,0,1,0,0
Homophobes,2,0,0,0,0,0,0,0
Sexist,3,0,0,0,0,12,0,2
Frenemies,1,0,0,0,0,0,0,0
Cryptic,5,0,0,0,0,2,0,3


In [62]:
confusion_1 = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_multi),
)

confusion_1.rename(columns = inv_cat_dic, inplace = True)
confusion_1.rename(index = inv_cat_dic, inplace = True)

print('Multinomial NB')
confusion_1

Multinomial NB


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic
Dunces,11,0,0,0,0,0,0,1
Criminals,2,0,2,0,0,0,0,0
Deadbeats,1,0,15,0,0,0,0,0
Racists,0,0,1,0,0,0,0,0
Homophobes,1,0,0,0,0,0,1,0
Sexist,12,0,0,0,0,5,0,0
Frenemies,1,0,0,0,0,0,0,0
Cryptic,6,0,1,0,0,0,0,3


In [63]:
confusion_2 = pd.DataFrame(
    data = confusion_matrix(y_test_mul, pred_bernoulli),
)

confusion_2.rename(columns = inv_cat_dic, inplace = True)
confusion_2.rename(index = inv_cat_dic, inplace = True)

print('Bernoulli NB')
confusion_2

Bernoulli NB


,Dunces,Criminals,Deadbeats,Racists,Homophobes,Sexist,Frenemies,Cryptic
Dunces,12,0,0,0,0,0,0,0
Criminals,2,0,2,0,0,0,0,0
Deadbeats,7,0,9,0,0,0,0,0
Racists,1,0,0,0,0,0,0,0
Homophobes,2,0,0,0,0,0,0,0
Sexist,17,0,0,0,0,0,0,0
Frenemies,1,0,0,0,0,0,0,0
Cryptic,10,0,0,0,0,0,0,0


In [64]:
from sklearn.metrics import precision_score

precision = precision_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

precision_1 = precision_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

precision_2 = precision_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression precision score: ', precision)
print('Multinomial NB precision score: ', precision_1)
print('Bernulli NB precision score: ', precision_2)

Log Regression precision score:  0.5565020210982945
Multinomial NB precision score:  0.6510147918816649
Bernulli NB precision score:  0.2517482517482518


In [65]:
from sklearn.metrics import recall_score

# recall = recall_score(
#     classifier['actual'],
#     classifier['predicted'],
#     pos_label = 'A'
# )

recall = recall_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

recall_1 = recall_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

recall_2 = recall_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression recall score: ', recall)
print('Multinomial NB recall score: ', recall_1)
print('Bernulli NB recall score: ', recall_2)

Log Regression recall score:  0.5714285714285714
Multinomial NB recall score:  0.5396825396825397
Bernulli NB recall score:  0.3333333333333333


In [66]:
from sklearn.metrics import f1_score

# f1_score(
#     classifier['actual'],
#     classifier['predicted'],
#     pos_label = 'A'
# )

f1_score_0 = f1_score(
    y_test_mul,
    pred_log_regr,
    pos_label = 2,
    average = 'weighted'
)

f1_score_1 = f1_score(
    y_test_mul,
    pred_multi,
    pos_label = 2,
    average = 'weighted'
)

f1_score_2 = f1_score(
    y_test_mul,
    pred_bernoulli,
    pos_label = 2,
    average = 'weighted'
)

print('Log Regression f1 score: ', f1_score_0)
print('Multinomial NB f1 score: ', f1_score_1)
print('Bernulli NB f1 score: ', f1_score_2)

Log Regression f1 score:  0.5518273976257169
Multinomial NB f1 score:  0.4994667168580212
Bernulli NB f1 score:  0.24074074074074073


In [67]:
knn_robust = KNeighborsClassifier()
knn_robust.fit(scalar.transform(X_train), y_train_mul.values.ravel())
model_score_mul(knn_robust)

I got good news, and bad news...
the bad news is that you will not like the score...
Now, the good news is that I'm right here to watch your face!!! LOL
The score for this algorithm is:  47.61904761904761 %

Your algorithm stinks so much, I could toss a coin and make better predictions =P...
